# 一次元移流拡散モデル（2017年度中級編）

## 本チュートリアルの内容

### モデル

2017年度中級編の課題であった一次元移流拡散モデル

\begin{equation}
\frac{\partial C}{\partial t} = -u\frac{\partial C}{\partial x} + \nu \frac{\partial^2 C}{\partial x^2} + W
\end{equation}

を対象とする。
- $C$ はモデルで計算するトレーサー濃度（初期値 $C=0$）
- $u$ は移流速度（$u=2.0$ で一定）
- $\nu$ は拡散係数（$\nu = 5.0$ で一定）
- $W$ は外力項であり，$0 < x < 60$ の範囲で，下式により与える。
\begin{equation}
W\left(x, t\right) = F\left(t\right) \sin\frac{\pi x}{60} = 
\left[\sin \frac{2\pi t}{120} + q\left(t\right)\right]\sin\frac{\pi x}{60}
\end{equation}
ここで、$q$ は未知の物理を表わすパラメータで，平均 0，標準偏差 1.0 のガウス乱数とする（演習ではデータファイルで与えられる）。

モデル領域は $0 \le x \le 400$ とし，格子間隔は $\Delta x = 10$ とする。時間ステップは $\Delta t=0.5$ とし，時間積分はリープフロッグ法により行う（計算モードを除去するため，15ステップに1度の頻度でオイラー法を用いる）。

### 真値、シミュレーションと疑似観測

- 真値<BR>
上記の条件で $t=600$ まで時間積分する。データの出力は 15 ステップ（7.5秒）に1回。
- シミュレーション<BR>
外力項のうち未知の物理 $q\left(t\right) = 0$ とする。データの出力は 15 ステップ（7.5秒）に1回。
- 疑似観測<BR>
同化計算に用いる疑似観測値は、$x = 160, 180, 200, 220, 240, 260, 280, 300$ において、187.5秒（375ステップ）から7.5秒（15ステップ）ごとに与える。疑似観測値は、真値に観測誤差として標準偏差 8.0 のガウス乱数を加えたものとする。

## プログラム

### [1] 必要なモジュールのインポート

In [ ]:
% matplotlib inline
% config InlineBackend.figure_format = 'svg'

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import interpolate
import advdiff
import os

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['mathtext.fontset'] = 'cm'

# 乱数シードの指定
np.random.seed(7)

### [2] モデルパラメータの設定

In [ ]:
params = {
    'nx': 40,
    'dx': 10.0,
    'dt': 0.5,
    'u': 2.0,
    'nu': 5.0,
    'freq_euler': 15
}

nt = 1200
freq_out = 15

### [2] 外力項を与える関数の設定

真値、シミュレーションの外力項はそれぞれ
\begin{align}
W_{\rm true}\left(x, t\right) &= \left[\sin \frac{2\pi t}{120} + q\left(t\right)\right]\sin\frac{\pi x}{60} \\
W_{\rm sim}\left(x, t\right) &= \sin \frac{2\pi t}{120} \sin\frac{\pi x}{60}
\end{align}
で計算する。$q\left(t\right)$ は 7.5 秒ごとの値を乱数により与え、その間の時刻における値は線形補間により与える。

In [ ]:
# 乱数データの生成
frc_noise = np.random.normal(loc=0.0, scale=1.0, size=81)
frc_noise.tofile('./data/noise/noise_frc.dat')
print('Avg.: {:>5.3f}'.format(np.mean(frc_noise)))
print('Std.: {:>5.3f}'.format(np.std(frc_noise)))

In [ ]:
# 外力項を与える関数の設定
# 時刻を引数として呼び出すと各格子点における外力の値を返す
x = np.arange(params['nx']) * params['dx']
t = np.arange(81) * params['dt'] * 15

#
frc_x = np.sin(np.pi * x / 60.) * np.where(x < 60., 1.,  0.)
frc_n = interpolate.interp1d(t, frc_noise, kind='linear')
frc_sim  = lambda t: np.sin(2. * np.pi * t / 120.) * frc_x
frc_true = lambda t: (np.sin(2. * np.pi * t / 120.) + frc_n(t)) * frc_x

### [3] 真値の計算

In [ ]:
nt = 1200
freq_out = 15
fmt_out = '%8.3f' * params['nx']

model = advdiff.Model(**params)

vt = [model.data]

for it in range(nt):
    _t = it * model.dt
    model.step_forward(force=frc_true(_t))
    if model.it % freq_out == 0:
        vt.append(model.data)

vt = np.array(vt)

np.savetxt('./data/state_true.dat', vt, fmt='%8.3f')

### [4] シミュレーションの計算

In [ ]:
nt = 1200
freq_out = 15
fmt_out = '%8.3f' * params['nx']

model = advdiff.Model(**params)

vs = [model.data]

for it in range(nt):
    _t = it * model.dt
    model.step_forward(force=frc_sim(_t))
    if model.it % freq_out == 0:
        vs.append(model.data)

vs = np.array(vs)

np.savetxt('./data/state_sim.dat', vs, fmt='%8.3f')

### [5] 疑似観測データの生成

In [ ]:
# 観測行列
H = np.zeros((8,40))
for i, j in enumerate([16, 18, 20, 22, 24, 26, 28, 30]):
    H[i,j] = 1.0
H @ model.x

# 真値からの観測
vo = vt[:,:] @ H.T

# 観測ノイズ
shape = vo.shape
size = shape[0] * shape[1]
obs_noise = np.random.normal(loc=0.0, scale=8.0, size=size).reshape(shape)
obs_noise.tofile('./data/noise/noise_obs.dat')
vo = vo + obs_noise
np.savetxt('./data/state_obs.dat', vo, fmt='%8.3f')

### [6] 描画

In [ ]:
it = 30

fig, ax = plt.subplots(figsize=(8,3))
plt.plot(x, vt[it,:], 'o-', c='C0', lw=1.0, label='True')
plt.plot(x, vs[it,:], 'o-', c='C1', lw=1.0, label='Simulation')
plt.plot(x @ H.T, vo[it,:], '^', c='C2', label='Observation')
plt.xlim(0, 400)
plt.ylim(-40, 40)
plt.xlabel(r'$x$ [km]')
plt.ylabel(r'$C$')
plt.legend(loc='lower right')
plt.grid(ls='--', c='0.5', lw=0.3)
plt.title(r'$t={:4.1f}$'.format(it * 7.5))
plt.tight_layout()